In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

# Setting warnings to ignore for cleaner output if expected warnings occur (e.g., from pd.get_dummies)
warnings.filterwarnings('ignore')

# Load the datasets
try:
    kouhaku_performers = pd.read_csv('kouhaku_performers_full_2015-2025.csv')
    artist_data = pd.read_csv('2artist_data.csv') # Renamed for clarity
    artist_master = pd.read_csv('artist_master_categorized_final.csv')
    # This file seems to be identical to 2artist_data.csv based on previous error outputs.
    # We will prioritize artist_data for features and clarify this to the user.
    spotify_popularity_dummy = pd.read_csv('spotify_popularity_cleaned.csv')
    analysis_clusters = pd.read_csv('1_analysis_clusters.csv')
except FileNotFoundError as e:
    print(f"エラー: ファイルが見つかりません - {e}. 全てのCSVファイルが正しいディレクトリにあるか確認してください。")
    exit()

print("--- データ読み込み完了 ---")

# --- Debug: Print columns of analysis_clusters and max year in artist_data ---
print("\n--- 1_analysis_clusters.csv の列名 ---")
print(analysis_clusters.columns)

print(f"\n--- artist_data.csv (特徴量データ) の最大年: {artist_data['year'].max()} ---")

# 補足: spotify_popularity_cleaned.csvのデータは、artist_data.csvのspotify関連列と類似しているようです。
# 今回は、artist_data.csvのspotify_followersとspotify_popularityを主要なSpotify指標として使用します。

# --- 1. データ統合と特徴量エンジニアリング ---

# 年ごとのアーティストの総テレビ出演回数を集計 (2artist_dataから)
artist_tv_appearances = artist_data.groupby(['year', 'artist'])['appearances'].sum().reset_index()
artist_tv_appearances.rename(columns={'appearances': 'total_tv_appearances_yearly'}, inplace=True)

# 主要音楽番組ごとの出演回数を集計
major_programs = [
    'NHK紅白歌合戦', 'ベストアーティスト', 'ミュージックステーションスーパーライブ',
    'CDTVライブ！ライブ！', 'FNS歌謡祭', 'Mステ ウルトラSUPER LIVE', 'COUNT DOWN TV', 'ベストヒット歌謡祭',
    'SONGS', 'バズリズム02', 'うたコン'
]

# Create pivot table for specific program appearances
program_appearances = artist_data.pivot_table(
    index=['year', 'artist'],
    columns='program',
    values='appearances',
    aggfunc='sum'
).fillna(0).reset_index()

# Clean program names for column renaming
cleaned_program_cols = {}
for col in program_appearances.columns:
    if col in ['year', 'artist']:
        continue
    cleaned_col_name = f'appearances_{col.replace(" ", "_").replace("！", "").replace("（", "").replace("）", "").replace("、", "").replace("・", "").replace("!", "").replace("?", "").replace("〜", "").replace("・", "_").replace("」", "").replace("「", "")}'
    cleaned_program_cols[col] = cleaned_col_name

program_appearances.rename(columns=cleaned_program_cols, inplace=True)


# Merge artist_tv_appearances and program_appearances
features_df = pd.merge(artist_tv_appearances, program_appearances, on=['year', 'artist'], how='left')

# artist_master_categorized_final.csv と結合 (artist_category, type, origin を追加)
cols_to_merge_from_master = ['artist', 'artist_category', 'type', 'origin']
existing_master_cols = [col for col in cols_to_merge_from_master if col in artist_master.columns]
if 'Gender' in artist_master.columns:
    existing_master_cols.append('Gender')

features_df = pd.merge(features_df, artist_master[existing_master_cols], on='artist', how='left')


# 2artist_data.csv からアーティストごとのspotify_followersとspotify_popularity（アーティストレベル）を取得
artist_spotify_by_year = artist_data[['year', 'artist', 'spotify_followers', 'spotify_popularity']].dropna(subset=['spotify_followers', 'spotify_popularity']).drop_duplicates(subset=['year', 'artist'], keep='first')

# 結合: features_dfにspotify_followersとspotify_popularity（アーティストレベル）をマージ
features_df = pd.merge(features_df, artist_spotify_by_year, on=['year', 'artist'], how='left')


# analysis_clusters.csv と結合 (cluster_labelの存在チェックを追加)
if 'cluster' in analysis_clusters.columns: # 'cluster_label'ではなく'cluster'
    features_df = pd.merge(features_df, analysis_clusters[['artist', 'cluster']], on='artist', how='left')
    features_df.rename(columns={'cluster': 'cluster_label'}, inplace=True) # 列名を'cluster_label'に統一
    print("info: 'cluster' 列が見つかりました。'cluster_label'として特徴量に結合します。")
elif 'cluster_label' in analysis_clusters.columns: # 念のため既存の'cluster_label'もチェック
    features_df = pd.merge(features_df, analysis_clusters[['artist', 'cluster_label']], on='artist', how='left')
    print("info: 'cluster_label' 列が見つかりました。特徴量として結合します。")
else:
    print("warning: 'cluster' 列も 'cluster_label' 列も '1_analysis_clusters.csv' に見つかりませんでした。この特徴量はモデルには含まれません。")

# --- 修正点: データの重複を削除し、(year, artist)のユニークな組み合わせを保証 ---
print("\n--- データの重複（year, artist）を削除中 ---")
initial_rows = len(features_df)
features_df.drop_duplicates(subset=['year', 'artist'], inplace=True)
print(f"初期行数: {initial_rows}, 重複削除後行数: {len(features_df)}")


# --- 2. ターゲット変数 'is_kohaku_performer' の作成 ---

# 紅白出場アーティストのセットを作成
kohaku_artists_by_year = kouhaku_performers.groupby('year')['artist'].apply(set).to_dict()

# features_df の各行について、紅白出場者かどうかを判定
features_df['is_kohaku_performer'] = 0 # デフォルトは非出場

for index, row in features_df.iterrows():
    year = row['year']
    artist = row['artist']
    if year in kohaku_artists_by_year and artist in kohaku_artists_by_year[year]:
        features_df.at[index, 'is_kohaku_performer'] = 1

print("\n--- データ統合とターゲット変数作成完了 ---")
print("is_kohaku_performer の分布:")
print(features_df['is_kohaku_performer'].value_counts())

print("\n統合済みデータフレームの先頭:")
print(features_df.head())
print("\n統合済みデータフレームの情報:")
print(features_df.info())


# --- 3. 特徴量エンジニアリングと前処理 ---

# 欠損値の確認と処理
print("\n--- 欠損値の確認 ---")
print(features_df.isnull().sum()[features_df.isnull().sum() > 0])

# 数値特徴量の欠損値を埋める（例: 中央値や平均値、または0）
numerical_cols = features_df.select_dtypes(include=np.number).columns.tolist()
if 'is_kohaku_performer' in numerical_cols:
    numerical_cols.remove('is_kohaku_performer')

for col in numerical_cols:
    features_df[col] = features_df[col].fillna(0) # 欠損値を0で埋める

# カテゴリカル特徴量の処理 (One-Hot Encoding, Label Encoding)
categorical_cols = features_df.select_dtypes(include='object').columns.tolist()
if 'artist' in categorical_cols:
    categorical_cols.remove('artist')

# cluster_labelが存在する場合のみLabelEncoderを適用
if 'cluster_label' in features_df.columns and features_df['cluster_label'].dtype == 'object':
    le = LabelEncoder()
    features_df['cluster_label'] = le.fit_transform(features_df['cluster_label'].fillna('Unknown'))
    if 'cluster_label' in categorical_cols:
        categorical_cols.remove('cluster_label')
elif 'cluster_label' not in features_df.columns:
    print("warning: 'cluster_label' 列が最終的な特徴量データに含まれていません。")


features_df = pd.get_dummies(features_df, columns=categorical_cols, dummy_na=False)

print("\n--- 欠損値処理とカテゴリカル特徴量エンコーディング完了 ---")
print("前処理後のデータフレームの先頭:")
print(features_df.head())
print("\n前処理後のデータフレームの情報:")
print(features_df.info())


# --- 4. 機械学習モデルの訓練、評価、予測 ---

# 目的変数 (y) と特徴量 (X) の定義
X = features_df.drop(['artist', 'is_kohaku_performer'], axis=1) # artistはIDなので除外
y = features_df['is_kohaku_performer']

# Xとyの列が揃っていることを確認
if X.empty or X.shape[1] == 0:
    print("\nエラー: 特徴量データ (X) が空であるか、列がありません。データ統合または特徴量エンジニアリングに問題がある可能性があります。")
    print("features_dfの最終的な状態を確認してください。")
    exit()


# 学習データ (2015-2023) と予測データ (2024) の分割
X_train_val = X[X['year'] <= 2023]
y_train_val = y[X['year'] <= 2023]

X_2024 = X[X['year'] == 2024]
artists_2024 = features_df[features_df['year'] == 2024]['artist']
y_2024_true = y[X['year'] == 2024]

# year列は学習に含めない
X_train_val = X_train_val.drop('year', axis=1)
if not X_2024.empty:
    X_2024_pred = X_2024.drop('year', axis=1)
else:
    X_2024_pred = pd.DataFrame(columns=X_train_val.columns) # 空のDataFrameを作成


# 訓練データが空でないことを確認
if X_train_val.empty or y_train_val.empty:
    print("\nエラー: 訓練データが空です。2015-2023年のデータが存在しないか、適切に分割されていません。")
    exit()

X_train = X_train_val
y_train = y_train_val


# 特徴量のスケーリング (数値特徴量のみ)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)


print("\n--- 機械学習モデルの訓練と評価 ---")

# モデルの訓練と評価は訓練データが存在する場合のみ実行
if not X_train.empty and not y_train.empty:
    # ロジスティック回帰モデル
    print("\n=== ロジスティック回帰モデル ===")
    log_reg = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', max_iter=1000)
    log_reg.fit(X_train_scaled, y_train)

    if not X_2024_pred.empty:
        # 列の整合性を確保
        missing_cols_2024 = set(X_train.columns) - set(X_2024_pred.columns)
        for c in missing_cols_2024:
            X_2024_pred[c] = 0
        X_2024_pred = X_2024_pred[X_train.columns] # 訓練データと同じ列順に並べ替え
        X_2024_pred_scaled = scaler.transform(X_2024_pred)
        X_2024_pred_scaled = pd.DataFrame(X_2024_pred_scaled, columns=X_2024_pred.columns, index=X_2024_pred.index)

        # 2024年の予測と評価
        y_pred_2024_log_reg = log_reg.predict(X_2024_pred_scaled)
        y_proba_2024_log_reg = log_reg.predict_proba(X_2024_pred_scaled)[:, 1]

        print("\n2024年 紅白出場予測 (ロジスティック回帰) 評価:")
        print(f"Accuracy: {accuracy_score(y_2024_true, y_pred_2024_log_reg):.4f}")
        print(f"Precision: {precision_score(y_2024_true, y_pred_2024_log_reg):.4f}")
        print(f"Recall: {recall_score(y_2024_true, y_pred_2024_log_reg):.4f}")
        print(f"F1-Score: {f1_score(y_2024_true, y_pred_2024_log_reg):.4f}")
        print(f"ROC AUC: {roc_auc_score(y_2024_true, y_proba_2024_log_reg):.4f}")
        print("Confusion Matrix:")
        print(confusion_matrix(y_2024_true, y_pred_2024_log_reg))

        print("\n2024年 紅白出場予測 (ロジスティック回帰) 結果 (予測確率降順):")
        log_reg_2024_results = pd.DataFrame({
            'artist': artists_2024,
            'predicted_proba': y_proba_2024_log_reg,
            'predicted_is_performer': y_pred_2024_log_reg,
            'true_is_performer': y_2024_true.reset_index(drop=True)
        }).sort_values(by='predicted_proba', ascending=False)
        print(log_reg_2024_results.head(10))
    else:
        print("2024年の予測対象データが存在しないため、ロジスティック回帰の2024年評価はスキップされました。")


    # ランダムフォレストモデル
    print("\n=== ランダムフォレストモデル ===")
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    rf_clf.fit(X_train_scaled, y_train)

    if not X_2024_pred.empty:
        # 2024年の予測と評価
        y_pred_2024_rf = rf_clf.predict(X_2024_pred_scaled)
        y_proba_2024_rf = rf_clf.predict_proba(X_2024_pred_scaled)[:, 1]

        print("\n2024年 紅白出場予測 (ランダムフォレスト) 評価:")
        print(f"Accuracy: {accuracy_score(y_2024_true, y_pred_2024_rf):.4f}")
        print(f"Precision: {precision_score(y_2024_true, y_pred_2024_rf):.4f}")
        print(f"Recall: {recall_score(y_2024_true, y_pred_2024_rf):.4f}")
        print(f"F1-Score: {f1_score(y_2024_true, y_pred_2024_rf):.4f}")
        print(f"ROC AUC: {roc_auc_score(y_2024_true, y_proba_2024_rf):.4f}")
        print("Confusion Matrix:")
        print(confusion_matrix(y_2024_true, y_pred_2024_rf))

        print("\n2024年 紅白出場予測 (ランダムフォレスト) 結果 (予測確率降順):")
        rf_2024_results = pd.DataFrame({
            'artist': artists_2024,
            'predicted_proba': y_proba_2024_rf,
            'predicted_is_performer': y_pred_2024_rf,
            'true_is_performer': y_2024_true.reset_index(drop=True)
        }).sort_values(by='predicted_proba', ascending=False)
        print(rf_2024_results.head(10))
    else:
        print("2024年の予測対象データが存在しないため、ランダムフォレストの2024年評価はスキップされました。")


    # 2025年の予測は、データが存在する場合のみ実行
    # ここでは2025年の特徴量データが存在しないため、警告メッセージを出力してスキップ
    print("\n--- 2025年 紅白出場アーティスト予測 ---")
    print(f"警告: 2025年の特徴量データがありません（artist_data.csvの最大年: {artist_data['year'].max()}）。2025年の予測はスキップされます。")

else:
    print("訓練データが空のため、モデルの訓練と予測は実行されませんでした。")


# --- データギャップと今後の課題の再確認 ---
print("\n--- データギャップと今後の課題 ---")
print("1. **YouTubeデータ**: 現在のデータセットにはYouTubeの再生回数やチャンネル登録者数、トレンド入り情報などが含まれていません。これは紅白選考の重要な要素と考えられるため、モデルの予測精度を高める上で不足しています。")
print("   -> 解決策: YouTube Data API v3からのデータ収集が必要ですが、APIキーの取得と、過去の時系列データ（特に年の発表直前までの伸び率）の収集は自動化が難しい場合があります（繰り返しAPIコールを行う、または外部データソースを利用）。")
print("2. **売上データ（CD/デジタル）**: オリコンやBillboard Japanなどの売上データは、人気と実績を示す重要な指標ですが、現在のデータセットには含まれていません。")
print("   -> 解決策: 外部の売上データを手動またはスクレイピングで収集する必要があります。")
print("3. **デジタル人気指標の時系列性**: 提供されたSpotifyデータは現時点のスナップショットである可能性が高く、特定の年の11月までの人気動向を正確に反映しているとは限りません。")
print("   -> 解決策: 各年の月次・週次の人気度データ（ストリーム数、ランキングなど）を別途収集し、発表直前までのトレンドや伸び率を特徴量とする必要があります。")
print("4. **紅白選考の定性的な要素**: NHKの選考には「その年の顔」「話題性」「バランス」といった定性的な基準も含まれます。これらを定量的な特徴量に落とし込むのは困難です。")
print("   -> 解決策: モデルは定量的なデータから学習するため、これらの要素は完全にモデル化することはできませんが、ニュース記事の感情分析など高度なNLPを用いることで一部取り込める可能性はあります。")
print("5. **ネガティブサンプルの網羅性**: 今回の分析では`2artist_data.csv`に含まれるアーティストを非出場候補としました。これが紅白選考で考慮される『全候補者』を網羅しているわけではない点もご留意ください。より包括的な候補者リストが必要な場合は、別のデータ収集が必要です。")

--- データ読み込み完了 ---

--- 1_analysis_clusters.csv の列名 ---
Index(['artist', 'total_tv_appearances', 'spotify_popularity',
       'youtube_hit_appearances', 'google_trends_score', 'cluster'],
      dtype='object')

--- artist_data.csv (特徴量データ) の最大年: 2024 ---
info: 'cluster' 列が見つかりました。'cluster_label'として特徴量に結合します。

--- データの重複（year, artist）を削除中 ---
初期行数: 7498, 重複削除後行数: 947

--- データ統合とターゲット変数作成完了 ---
is_kohaku_performer の分布:
is_kohaku_performer
0    546
1    401
Name: count, dtype: int64

統合済みデータフレームの先頭:
    year     artist  total_tv_appearances_yearly  appearances_NHK紅白歌合戦  \
0   2015        AAA                          6.0                   6.0   
3   2015         AI                          0.0                   0.0   
15  2015      AKB48                          8.0                   8.0   
34  2015  BABYMETAL                          0.0                   0.0   
38  2015   BREAKERZ                          0.0                   0.0   

    appearances_ベストアーティスト  appearances_ミュージックステーションスーパ

ValueError: array length 109 does not match index length 218

In [10]:
pip install spotipy pytrends scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pytrends.request import TrendReq
from tqdm import tqdm
import time
import os
import pickle

# === 🔐 Spotify 認証情報 ===
SPOTIFY_CLIENT_ID = '2892e96c25394749aee2a940363bdbc3'
SPOTIFY_CLIENT_SECRET = '4afb990292a24641a522c2adb908b158'

# === 📥 CSVファイル読み込み ===
df = pd.read_csv("kouhaku_performers_full_2015-2025.csv")
df["artist"] = df["artist"].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df["appearances"] = pd.to_numeric(df["appearances"], errors='coerce').fillna(0)
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# 年ごとのアーティスト集合
year_artist = df.groupby("year")["artist"].apply(set).to_dict()
all_artists = sorted(set(df[df["year"] < 2024]["artist"].dropna().unique()))

# === API 初期化 ===
spotify_auth = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=spotify_auth)
pytrends = TrendReq(hl='ja-JP', tz=540)

# === 🔄 キャッシュ辞書 ===
spotify_cache = {}
trend_cache = {}

# キャッシュ保存関数（後から再利用可能にする）
def save_cache():
    with open("spotify_cache.pkl", "wb") as f:
        pickle.dump(spotify_cache, f)
    with open("trend_cache.pkl", "wb") as f:
        pickle.dump(trend_cache, f)

# キャッシュ読み込み
def load_cache():
    global spotify_cache, trend_cache
    if os.path.exists("spotify_cache.pkl"):
        with open("spotify_cache.pkl", "rb") as f:
            spotify_cache = pickle.load(f)
    if os.path.exists("trend_cache.pkl"):
        with open("trend_cache.pkl", "rb") as f:
            trend_cache = pickle.load(f)

load_cache()

# === 外部データ取得 ===
def get_spotify_features(artist):
    if artist in spotify_cache:
        return spotify_cache[artist]
    try:
        results = sp.search(q=f"artist:{artist}", type='artist', limit=1)
        items = results['artists']['items']
        if items:
            popularity = items[0]['popularity']
            followers = items[0]['followers']['total']
        else:
            popularity, followers = 0, 0
    except:
        popularity, followers = 0, 0
    spotify_cache[artist] = (popularity, followers)
    return popularity, followers

def get_trend_score(artist, year):
    key = f"{artist}_{year}"
    if key in trend_cache:
        return trend_cache[key]
    try:
        pytrends.build_payload([artist], timeframe=f'{year}-01-01 {year}-12-31')
        data = pytrends.interest_over_time()
        if not data.empty:
            score = data[artist].mean()
        else:
            score = 0
    except:
        score = 0
    trend_cache[key] = score
    return score

# === 学習データ作成 ===
features, labels = [], []
years_train = range(2016, 2024)
print("🧪 学習データ作成中...")
for target_year in tqdm(years_train):
    prev_years = list(range(2015, target_year))
    current_set = year_artist.get(target_year, set())

    for artist in all_artists:
        appeared_prev = int(any(artist in year_artist.get(y, set()) for y in prev_years))
        recent_appearances = sum(artist in year_artist.get(y, set()) for y in prev_years[-3:])
        trend_score = get_trend_score(artist, target_year - 1)
        popularity, followers = get_spotify_features(artist)

        features.append([
            appeared_prev,
            recent_appearances,
            trend_score,
            popularity,
            followers
        ])
        labels.append(int(artist in current_set))
        time.sleep(0.1)  # 軽く制限回避

# === モデル学習 ===
X_train = pd.DataFrame(features, columns=["appeared_prev", "recent_appearances", "trend_score", "popularity", "followers"])
y_train = pd.Series(labels)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === 2024年の検証 ===
print("\n🔍 2024年の予測と正誤評価...")
X_2024, y_2024, artists_2024 = [], [], []

for artist in tqdm(all_artists):
    appeared_prev = int(any(artist in year_artist.get(y, set()) for y in range(2015, 2024)))
    recent_appearances = sum(artist in year_artist.get(y, set()) for y in range(2021, 2024))
    trend_score = get_trend_score(artist, 2023)
    popularity, followers = get_spotify_features(artist)

    X_2024.append([appeared_prev, recent_appearances, trend_score, popularity, followers])
    y_2024.append(int(artist in year_artist.get(2024, set())))
    artists_2024.append(artist)
    time.sleep(0.1)

X_2024_df = pd.DataFrame(X_2024, columns=["appeared_prev", "recent_appearances", "trend_score", "popularity", "followers"])
y_2024_pred = model.predict(X_2024_df)

print(classification_report(y_2024, y_2024_pred, target_names=["Not Appeared", "Appeared"]))

# === 2025年の予測 ===
print("\n🔮 2025年出演予測...")
X_2025, artists_2025 = [], []

for artist in tqdm(all_artists):
    appeared_prev = int(any(artist in year_artist.get(y, set()) for y in range(2015, 2025)))
    recent_appearances = sum(artist in year_artist.get(y, set()) for y in range(2022, 2025))
    trend_score = get_trend_score(artist, 2024)
    popularity, followers = get_spotify_features(artist)

    X_2025.append([appeared_prev, recent_appearances, trend_score, popularity, followers])
    artists_2025.append(artist)
    time.sleep(0.1)

X_2025_df = pd.DataFrame(X_2025, columns=["appeared_prev", "recent_appearances", "trend_score", "popularity", "followers"])
y_2025_pred = model.predict(X_2025_df)
y_2025_prob = model.predict_proba(X_2025_df)[:, 1]  # 出演確率

# 出演予測アーティスト一覧（スコア付き）
predicted_df = pd.DataFrame({
    "artist": artists_2025,
    "predicted": y_2025_pred,
    "probability": y_2025_prob
})

predicted_2025 = predicted_df[predicted_df["predicted"] == 1].sort_values(by="probability", ascending=False)

print("\n 2025年 紅白出演予測アーティスト（上位候補）:")
print(predicted_2025[["artist", "probability"]].head(20))

# CSV出力（任意）
predicted_2025.to_csv("predicted_kouhaku_2025.csv", index=False)
print("\n 出力完了: predicted_kouhaku_2025.csv")

# キャッシュ保存
save_cache()


🧪 学習データ作成中...


100%|██████████| 8/8 [28:46<00:00, 215.83s/it]



🔍 2024年の予測と正誤評価...


100%|██████████| 198/198 [02:20<00:00,  1.41it/s]


              precision    recall  f1-score   support

Not Appeared       0.92      0.94      0.93       169
    Appeared       0.62      0.55      0.58        29

    accuracy                           0.88       198
   macro avg       0.77      0.75      0.76       198
weighted avg       0.88      0.88      0.88       198


🔮 2025年出演予測...


 30%|██▉       | 59/198 [00:40<01:31,  1.52it/s]